censusdata docs: https://jtleider.github.io/censusdata/example1.html</br>
for la county code: https://notary.cdn.sos.ca.gov/forms/notary-county-codes.pdf</br>
example of censustract package: https://towardsdatascience.com/mapping-us-census-data-with-python-607df3de4b9c


In [1]:
import pandas as pd
import numpy as np 

import censusdata
import censusgeocode as cg

from time import time
import threading

In [2]:
cd

/Users/karatechop


In [3]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)


In [23]:
def search_census_concepts(search_term):
    """Finds relevant census topic call codes
    based on a given search term input."""
    
    sample = censusdata.search('acs1', 2018,'concept', search_term)
    return sample

search_census_concepts('DETAILED RACE')

[('B02003_001E', 'DETAILED RACE', 'Estimate!!Total'),
 ('B02003_002E', 'DETAILED RACE', 'Estimate!!Total!!Population of one race'),
 ('B02003_003E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of one race!!White alone'),
 ('B02003_004E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of one race!!Black or African American alone'),
 ('B02003_005E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of one race!!American Indian and Alaska Native alone'),
 ('B02003_006E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of one race!!Asian alone'),
 ('B02003_007E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone'),
 ('B02003_008E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of one race!!Some other race alone'),
 ('B02003_009E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of two or more races'),
 ('B02003_010E',
  'DETAILED RACE',
  'Estimate!!Total!!Population of two or more races!!Population of two races

In [81]:
tract_pops = censusdata.download('acs5', 
             2019,
             censusdata.censusgeo([('state', '06'), ('county', '037'), ('tract','*')]),
             ['B01003_001E','B25001_001E','B19113_001E']).reset_index(drop=False)\
             .rename(columns={'index':'tract',
                              'B01003_001E':'tract_pop',
                              'B25001_001E':'housing_units',
                              'B19113_001E':'med_income_12mo_adj'})
tract_pops['tract'] = [str(i)[13:-88] for i in tract_pops['tract']]
tract_pops

,tract,tract_pop,housing_units,med_income_12mo_adj
0,4827.02,2373,845,100114
1,5002.01,7267,2251,127568
2,5002.02,4988,1567,152742
3,5003,2973,1108,132386
4,5005,2703,931,60625
...,...,...,...,...
2341,1041.08,6001,1483,51690
2342,1042.03,5441,1184,48516
2343,1042.04,3679,786,53636
2344,1044.03,3341,710,62337


In [82]:
tract_pops.tract_pop.describe()

count     2346.000000
mean      4297.344416
std       1587.065962
min          0.000000
25%       3203.750000
50%       4211.500000
75%       5314.000000
max      12463.000000
Name: tract_pop, dtype: float64

In [83]:
low_high = [(4278 - 1556),(4278 + 1556)]

tract_pops.loc[tract_pops['tract_pop']<low_high[0],'pop_level']='low'
tract_pops.loc[tract_pops['tract_pop']>low_high[1],'pop_level']='high'
tract_pops['pop_level'].fillna('mid',inplace=True)

tract_pops

,tract,tract_pop,housing_units,med_income_12mo_adj,pop_level
0,4827.02,2373,845,100114,low
1,5002.01,7267,2251,127568,high
2,5002.02,4988,1567,152742,mid
3,5003,2973,1108,132386,mid
4,5005,2703,931,60625,low
...,...,...,...,...,...
2341,1041.08,6001,1483,51690,high
2342,1042.03,5441,1184,48516,mid
2343,1042.04,3679,786,53636,mid
2344,1044.03,3341,710,62337,mid


In [26]:
race_info = censusdata.download('acs1', 
             2018,
             censusdata.censusgeo([('state', '06'), ('county', '38')]),
             ['B02003_001E']).reset_index(drop=False)

race_info

ValueError: Unexpected response (URL: https://api.census.gov/data/2018/acs/acs1?get=NAME,B02003_001E&for=county:38&in=state:06):  

## FIND TRACT FROM COORDS

In [15]:
df = pd.read_csv('la_lat_lon_expanded.csv')

In [16]:
len(df['coords'].unique())

31265

In [10]:
# %load_ext line_profiler
# %lprun -f coords_to_tracts coords_to_tracts()
# #             ^^^^^---- this call will be profiled
# #         ^^^-----------function to profile

In [17]:
df['tract'] = None

In [20]:
def coords_to_tracts_1():
    start = time()
    counter = 0    

    for i in df['coords'].unique()[:len(df['coords'].unique())//2]:
        counter+=1
        
        try:
            coords = i.split(' , ')
            tract = cg.coordinates(y=coords[0], x=coords[1])['2020 Census Blocks'][0]['TRACT'] 
            tract = tract[:-2] + '.' + tract[-2:]
            df.loc[df['coords'] == i,'tract'] = tract 
    
        except: 
            print(f'failed for coords {i}')
            pass
        
        if counter%500==0:
            unique = df['coords'].unique()
            print(f'{round((time()-start)/60,2)} minutes have passed, {unique} unique coords remaining.')
    
    print(f'that took {round((time()-start)/60,2)} minutes.')

In [21]:
def coords_to_tracts_2():
    start = time()
    counter = 0    

    for i in df['coords'].unique()[len(df['coords'].unique())//2:]:
        counter+=1
        
        try:
            coords = i.split(' , ')
            tract = cg.coordinates(y=coords[0], x=coords[1])['2020 Census Blocks'][0]['TRACT'] 
            tract = tract[:-2] + '.' + tract[-2:]
            df.loc[df['coords'] == i,'tract'] = tract 
    
        except: 
            print(f'failed for coords {i}')
            pass
        
        if counter%1000==0:
            print(f'coords_to_tracts_2 at {counter}.')
    
    print(f'that took {round((time()-start)/60,2)} minutes.')

In [38]:
# if __name__ =="__main__":
#     t1 = threading.Thread(target=coords_to_tracts_1)
#     t2 = threading.Thread(target=coords_to_tracts_2)
    
#     t1.start()
#     t2.start()
 
#     t1.join()
#     t2.join()
 
#     # both threads completely executed
#     print(f"\n\n\nBoth threads are done!\n\n{df['coords'].unique()}")
#     df.to_csv('la_lat_lon_tract_expanded')

In [76]:
backup = df
backup2 = df

In [84]:
stop_freq = df['tract'].value_counts().rename_axis('tract').reset_index(name='stop_freq')

In [92]:
tract_info = pd.merge(tract_pops,stop_freq)
low_high = [65,200]

tract_info.loc[tract_info['stop_freq']<low_high[0],'freq_level']='low'
tract_info.loc[tract_info['stop_freq']>low_high[1],'freq_level']='high'
tract_info['freq_level'].fillna('mid',inplace=True)

tract_info.columns

Index(['tract', 'tract_pop', 'housing_units', 'med_income_12mo_adj',
       'pop_level', 'stop_freq', 'freq_level'],
      dtype='object')

In [99]:
tract_info_next = pd.merge(df,tract_info[['tract','pop_level','freq_level']])
tract_info_next['date'] = pd.to_datetime(tract_info_next['date'])
tract_info_next.head()

,date,time,race,location,city,wave,mins_before_dusk,mins_stop_time,mins_dusk,dark,period,coords,tract,pop_level,freq_level
0,2018-07-01,1900-01-01 19:05:00,hispanic/latino(a),600 south alvarado st,Los Angeles,1,93.0,1145.0,1238.0,0,dusk,"34.059389 , -118.274837",2089.03,mid,high
1,2018-07-02,1900-01-01 17:30:00,black/african american,600 south alvarado st,Los Angeles,1,188.0,1050.0,1238.0,0,dusk,"34.059389 , -118.274837",2089.03,mid,high
2,2018-07-02,1900-01-01 19:20:00,hispanic/latino(a),600 south alvarado st,Los Angeles,1,78.0,1160.0,1238.0,0,dusk,"34.059389 , -118.274837",2089.03,mid,high
3,2018-07-02,1900-01-01 19:20:00,hispanic/latino(a),600 south alvarado st,Los Angeles,1,78.0,1160.0,1238.0,0,dusk,"34.059389 , -118.274837",2089.03,mid,high
4,2018-07-07,1900-01-01 19:34:00,hispanic/latino(a),south westlake avenue & west 6th street,Los Angeles,1,63.0,1174.0,1237.0,0,dusk,"34.0589914 , -118.2736649",2089.03,mid,high


In [124]:
import warnings

In [141]:
warnings.filterwarnings("ignore") 

tract_info_next.loc[tract_info_next['date'].dt.year == 2018,['dst1','dst2']] = pd.to_datetime('3/11/18'),pd.to_datetime('11/04/18')
tract_info_next.loc[tract_info_next['date'].dt.year == 2019,['dst1','dst2']] = pd.to_datetime('3/10/19'),pd.to_datetime('11/03/19')
tract_info_next.loc[tract_info_next['date'].dt.year == 2020,['dst1','dst2']] = pd.to_datetime('3/08/20'),pd.to_datetime('11/01/20')
tract_info_next.loc[tract_info_next['date'].dt.year == 2021,['dst1','dst2']] = pd.to_datetime('3/14/21'),pd.to_datetime('11/07/21')
tract_info_next.loc[tract_info_next['date'].dt.year == 2022,['dst1','dst2']] = pd.to_datetime('3/13/22'),pd.to_datetime('11/06/22')

tract_info_next['diff_dst1'] = (tract_info_next['date']-tract_info_next['dst1'])
tract_info_next['diff_dst1'] = [i.days for i in tract_info_next['diff_dst1']]

tract_info_next['diff_dst2'] = (tract_info_next['date']-tract_info_next['dst2'])
tract_info_next['diff_dst2'] = [i.days for i in tract_info_next['diff_dst2']]

tract_info_final = tract_info_next[((tract_info_next['diff_dst1'] <= 30) & (tract_info_next['diff_dst1'] >= -30))|\
                                   ((tract_info_next['diff_dst2'] <= 30) & (tract_info_next['diff_dst2'] >= -30))]

In [142]:
tract_info_next.shape

(149655, 19)

In [143]:
tract_info_final.shape

(44871, 19)

In [146]:
tract_info_final.to_csv('la_final_seasonal.csv',index=False)
tract_info_next.to_csv('la_final_non_seasonal.csv',index=False)
tract_info_next.shape

(149655, 19)